In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from copy import copy
import os 
import pandas as pd
import numpy as np

from torch.utils.data import DataLoader

from scaler import FeaturesScaler
from dataloader import SimpleMagni
from simple_mlp import LightPointPredictor

In [2]:
EXTRAS_PATH = "lhmp-thor-magni-challenge-extras"  # set path to package directory
DATA_PATH = os.path.join(EXTRAS_PATH, "data")  # path to data directory
SCENARIOS = ["Scenario_1", "Scenario_2", "Scenario_3", "Scenario_4", "Scenario_5"]

CHECKPOINTS_PATHS = {
    "Scenario_1": f"{EXTRAS_PATH}/logs/Scenario_1_simple_mlp/version_0/checkpoints/epoch=35-val_loss=0.85.ckpt",
    "Scenario_2": f"{EXTRAS_PATH}/logs/Scenario_2_simple_mlp/version_0/checkpoints/epoch=45-val_loss=0.87.ckpt",
    "Scenario_3": f"{EXTRAS_PATH}/logs/Scenario_3_simple_mlp/version_0/checkpoints/epoch=37-val_loss=0.89.ckpt",
    "Scenario_4": f"{EXTRAS_PATH}/logs/Scenario_4_simple_mlp/version_0/checkpoints/epoch=19-val_loss=0.81.ckpt",
    "Scenario_5": f"{EXTRAS_PATH}/logs/Scenario_5_simple_mlp/version_0/checkpoints/epoch=29-val_loss=0.66.ckpt",
}  # set path to checkpoints

HYPERPARAMETERS = {
    "lr": 1e-3,
    "bs": 32,
    "scheduler_patience": 5,
    "val_freq": 2,  # validation every n epochs
    "patience": 10,  # 10 epochs early stopping
}
DATA_CONFIG = {
    "inputs": ["speeds"],
    "obs_len": 8,
    "pred_len": 12,
    "output": "speeds",
}
NETWORK_CONFIG = {
    "hidden_units": [32, 16],
    "dropout": 0.2,
    "batch_norm": True,
    "activation": "prelu",
    "obs_len": DATA_CONFIG["obs_len"],
    "pred_len": 12,
    "n_features": 3,
}

### Helper Function

In [3]:
def get_test_dl_and_stats(test_scenario: str):
    train_scenarios = copy(SCENARIOS)
    train_scenarios.remove(test_scenario)
    train_tracklets = []
    for train_scenario in train_scenarios:
        df = pd.read_csv(
            os.path.join(DATA_PATH, train_scenario + ".csv"), index_col="Time"
        )
        tracklets = [group for _, group in df.groupby("tracklet_id")]
        train_tracklets.extend(tracklets)
    _, statistics = FeaturesScaler.scale_dataset(train_tracklets)
    test_df = df = pd.read_csv(
        os.path.join(DATA_PATH, test_scenario + ".csv"), index_col="Time"
    )
    test_tracklets = [group for _, group in test_df.groupby("tracklet_id")]
    test_ds = SimpleMagni(test_tracklets)
    test_dl = DataLoader(test_ds, HYPERPARAMETERS["bs"], shuffle=False)
    return test_dl, statistics

In [4]:
scenarios_predictions = []
for scenario in SCENARIOS:
    checkpt_path = CHECKPOINTS_PATHS[scenario]
    test_dl, statistics = get_test_dl_and_stats(scenario)
    DATA_CONFIG.update(dict(training_data_stats=statistics))
    model_pred = LightPointPredictor.load_from_checkpoint(
        checkpt_path,
        data_cfg=DATA_CONFIG,
        network_cfg=NETWORK_CONFIG,
        hyperparameters_cfg=HYPERPARAMETERS,
        map_location={"cuda:0": "cpu"},
    )
    model_pred.eval()
    scenario_predictions = []
    for batch_idx, batch in enumerate(test_dl):
        yhat = model_pred.predict_step(batch, batch_idx)
        scenario_predictions.append(yhat["y_hat"][0].numpy())
    scenarios_predictions.append(np.concatenate(scenario_predictions, axis=0))
scenarios_predictions = np.array(scenarios_predictions, dtype=object)
np.save(
    os.path.join(EXTRAS_PATH, "simple_mlp_predictions.npy"),
    scenarios_predictions,
    allow_pickle=True,
)